In [1]:
import datetime
print("This notebook is run in", datetime.date.today().strftime('%B %d, %Y'))

This notebook is run in June 13, 2024


In [15]:
from IPython.display import Image
import cv2
from rmn import RMN
import pandas as pd
import os
import numpy as np

In [18]:
def process_frames_for_emotion(input_folder, output_csv):
    data = []
    m = RMN()

    for subdir, dirs, files in os.walk(input_folder):
        for file in sorted(files):
            if file.endswith(".png"):
                image_path = os.path.join(subdir, file)
                image = cv2.imread(image_path)
                if image is None:
                    continue

                print(f"Processing file: {file}") 
                try:
                    frame_number = int(file.split('_')[1].split('.')[0])
                except ValueError:
                    print(f"Skipping file with unexpected name format: {file}")
                    continue

                infant_id = subdir.split('_')[-1]

                results = m.detect_emotion_for_single_frame(image)
                if results:
                    print(f"Infant ID: {infant_id}, Frame: {frame_number}, Num faces: {len(results)}")
                    emo_label = results[0]['emo_label']
                    
                    # extracting the probabilities
                    proba_dict = {list(prob.keys())[0]: list(prob.values())[0] for prob in results[0]['proba_list']}
                    data.append({
                        'infant_id': infant_id,
                        'frame_number': frame_number,
                        'emo_label': emo_label,
                        'angry': proba_dict.get('angry', 0),
                        'disgust': proba_dict.get('disgust', 0),
                        'fear': proba_dict.get('fear', 0),
                        'happy': proba_dict.get('happy', 0),
                        'sad': proba_dict.get('sad', 0),
                        'surprise': proba_dict.get('surprise', 0),
                        'neutral': proba_dict.get('neutral', 0)
                    })

                    # save the image with drawn results
                    image = m.draw(image, results)
                    output_image_path = os.path.join(subdir, f"output_{file}")
                    cv2.imwrite(output_image_path, image)
                else:
                    print(f"Infant ID: {infant_id}, Frame: {frame_number}, Num faces: 0")
                    # append a row with NaNs for probabilities and emotion label
                    data.append({
                        'infant_id': infant_id,
                        'frame_number': frame_number,
                        'emo_label': np.nan,
                        'angry': np.nan,
                        'disgust': np.nan,
                        'fear': np.nan,
                        'happy': np.nan,
                        'sad': np.nan,
                        'surprise': np.nan,
                        'neutral': np.nan
                    })

    df = pd.DataFrame(data)
    df.sort_values(by=['infant_id', 'frame_number'], inplace=True)
    df.to_csv(output_csv, index=False)

In [19]:
output_folder = "/workspaces/wiggle-face/frames_first_20_videos"
output_csv = "/workspaces/wiggle-face/labels_first_20_videos.csv" 
process_frames_for_emotion(output_folder, output_csv)

Processing file: frame_000000.png
num faces: 0
Infant ID: 000000, Frame: 0, Num faces: 0
Processing file: frame_000001.png
num faces: 1
Infant ID: 000000, Frame: 1, Num faces: 1
Processing file: frame_000002.png
num faces: 1
Infant ID: 000000, Frame: 2, Num faces: 1
Processing file: frame_000003.png
num faces: 0
Infant ID: 000000, Frame: 3, Num faces: 0
Processing file: frame_000004.png
num faces: 0
Infant ID: 000000, Frame: 4, Num faces: 0
Processing file: frame_000005.png
num faces: 0
Infant ID: 000000, Frame: 5, Num faces: 0
Processing file: frame_000006.png
num faces: 0
Infant ID: 000000, Frame: 6, Num faces: 0
Processing file: frame_000007.png
num faces: 0
Infant ID: 000000, Frame: 7, Num faces: 0
Processing file: frame_000008.png
num faces: 0
Infant ID: 000000, Frame: 8, Num faces: 0
Processing file: frame_000009.png
num faces: 0
Infant ID: 000000, Frame: 9, Num faces: 0
Processing file: frame_000010.png
num faces: 0
Infant ID: 000000, Frame: 10, Num faces: 0
Processing file: fra